In [39]:
from llama_index.llms.groq import Groq

In [40]:
import os
from llama_index.core import Settings
#from llama_index.llms.openai import OpenAI
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.response.pprint_utils import pprint_response

In [41]:
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
llm = Groq(temperature=0,model="llama3-70b-8192")
Settings.llm = llm


In [50]:
# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("youtubetranscript/zeeshanusmani").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

In [74]:
query_engine = index.as_query_engine(streaming=True, similarity_top_k=1)
response_stream = query_engine.query(
    "give me two million subscriber video link",
)
response_stream.print_response_stream()

There is no information in the provided context about a two million subscriber video link. The context only provides information about a video with one million subscribers.

In [64]:
query_engine = index.as_chat_engine(chat_mode="context", llm=llm,verbose=True)
response = query_engine.chat("give me two million subscriber video link")
print(response)

The YouTube link for the 2 million subscriber video is:

https://www.youtube.com/watch?v=wT6tjKarxRI


In [76]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
query_engine = index.as_chat_engine(chat_mode="condense_plus_context", llm=llm,
                                    context_prompt=(
                                        "You are a chatbot, able to have normal interactions, as well as talk"
                                        " about an Zeeshan usmani youtube channel videos. all the source documents contains json format consists"
                                        "source, title, description, publish_date, author, transcript, youtubelink"
                                        "Here are the relevant documents for the context:\n"
                                        "{context_str}"
                                        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user, and always return the youtubelink and publish_date from soruce document"
                                    )
                                    ,verbose=False,similarity_top_k=1)
response = query_engine.chat("How can i migrate from pakistan to dubai? give me youtube video link and publish date")
print(response)
pprint_response(response,show_source=True)


Migrating from Pakistan to Dubai can be a challenging process, but with the right guidance, it can be achievable.

According to Zeeshan Usmani's video, "Podcast with Sunny Ali", he shares valuable insights on how to migrate from Pakistan to Dubai. He emphasizes the importance of having a second passport and shares strategies on how to get immigration and a second passport.

Here's the YouTube video link: https://www.youtube.com/watch?v=Oc_hIqVkHvg

Publish Date: Not specified in the provided document, but I can try to help you find it if you need it!

In the video, Zeeshan Usmani also talks about the difficulties of doing business in Pakistan due to political instability and how it's affecting investments. He also highlights the importance of emerging technologies like AI, Data Sciences, and Machine Learning for a bright future.

If you have any specific questions about migrating from Pakistan to Dubai, feel free to ask, and I'll do my best to help!
Final Response: Migrating from Pakis

In [82]:
#query_engine = index.as_query_engine()
response = query_engine.chat("How many youtube subscribers of zishan usmani yotube channel? he told me that is his video he has 2 million subscribers?")
pprint_response(response,show_source=True)
print(response)

Final Response: I remember! According to Zeeshan Usmani's video, he
mentioned that he has 2.68 lakh (268,000) subscribers, and he
projected that he would reach 2.5 million subscribers within 3 years.
However, you mentioned that he updated his subscriber count to 2
million subscribers in a later video.
______________________________________________________________________
Source Node 1/1
Node ID: cd5ce9dc-83dc-4e85-aaf2-f6b6424368c3
Similarity: 0.8752598231185907
Text: The average number of views I used to get per day was 165000
people watching my videos, currently they are watching 244000, it
means more people are watching my videos than my subscribers, then you
click on the interesting thing, you can see the future projection.
Click on how much the channel Majeed will grow. Here future
projections are made t...
I remember! According to Zeeshan Usmani's video, he mentioned that he has 2.68 lakh (268,000) subscribers, and he projected that he would reach 2.5 million subscribers within 3

In [83]:
response = query_engine.chat("give me all youtube channel videos of zeeshan usmani in table foramt?")
#pprint_response(response,show_source=True)
print(response)

I apologize, but I don't have access to the entire list of Zeeshan Usmani's YouTube channel videos. The context provided only includes one video transcript.

However, I can provide you with a sample table format for Zeeshan Usmani's YouTube channel videos:

| **Video Title** | **Description** | **Publish Date** | **Author** | **Transcript** | **YouTube Link** |
| --- | --- | --- | --- | --- | --- |
| ریحان اللہ والاسے چبھتے سوالات | ... | ... | Zeeshan Usmani | ... | https://www.youtube.com/watch?v=bETjYxNIsLk |
| ... | ... | ... | Zeeshan Usmani | ... | ... |
| ... | ... | ... | Zeeshan Usmani | ... | ... |

Please note that I only have access to the one video transcript provided in the context. If you provide me with the entire dataset of Zeeshan Usmani's YouTube channel videos, I can help you populate this table.

Let me know if there's anything else I can assist you with!


In [54]:
response = query_engine.query("what is price of success?")
pprint_response(response,show_source=True)
print(response)

Final Response: According to the transcript, the price of success is
the costs that successful people pay, which are not visible, such as
hard work, sacrifices, and tiredness. They have to pay the price of
their success, which includes their time, relationships, and personal
life. They have to work hard, miss important events, and sacrifice
their personal life to achieve success.
______________________________________________________________________
Source Node 1/2
Node ID: 939e22b2-2bb9-4ab5-aacd-aee28d773aad
Similarity: 0.8374664317783173
Text: {     "source": "qvnjim6o8Ik",     "title": "Price of Success |
کامیابی کی قیمت",     "description": "Unknown",     "view_count":
18521,     "thumbnail_url":
"https://i.ytimg.com/vi/qvnjim6o8Ik/hq720.jpg?v=65f4c4d6",
"publish_date": "2022-06-15 00:00:00",     "length": 264,
"author": "Zeeshan Usmani",     "transcript": "Assalam alekum, m...
______________________________________________________________________
Source Node 2/2
Node ID: 2be2a327

In [72]:
response = query_engine.chat("give me the summary which was publish date on 2022-01-19?")
pprint_response(response,show_source=True)
print(response)

Final Response: I think you're referring to the video "Happy New Year
2022 | نیا سال مبارک" by Zeeshan Usmani.  Here's a summary of the
video:  Zeeshan Usmani starts by wishing his viewers a Happy New Year.
He then reflects on how people often focus on the negative aspects of
the world, such as corruption, oppression, and injustice. However, he
suggests that instead of complaining, we should take responsibility
for our own actions and make a positive impact.  He criticizes people
who borrow views from others, without having original ideas or taking
action. He also pokes fun at motivational speakers who use dramatic
gestures and expressions to engage their audience, but lack substance.
Zeeshan Usmani emphasizes the importance of self-reflection, taking
responsibility, and making a positive difference in the world.
**Publish Date:** 2021-12-20 (not 2022-01-19, I apologize for the
mistake)  Let me know if you'd like more information!
_______________________________________________________

In [73]:
response = query_engine.chat("this is wrong info there is video available which publisg date is 2022-01-19?")
pprint_response(response,show_source=True)
print(response)

Final Response: I apologize for the mistake. I don't have information
about a specific video published on 2022-01-19. The transcripts I have
access to are from two videos:  1. "Meeting with Mufti Tariq Masood |
مفتی طارق مسعود سے ملاقات | Zeeshan Usmani" 2. "What Pakistan Should
Do پاکستان کیا کرے | Zeeshan Usmani"  If you can provide more context
or information about the video published on 2022-01-19, I'll try to
help you find the summary.
______________________________________________________________________
Source Node 1/2
Node ID: e360f9c8-3733-4fbc-9569-781df30b6278
Similarity: 0.8317222791567985
Text: {     "source": "u0-krlmX1_0",     "title": "Meeting with Mufti
Tariq Masood | مفتی طارق مسعود سے ملاقات | Zeeshan Usmani",
"description": "Unknown",     "view_count": 389959,
"thumbnail_url":
"https://i.ytimg.com/vi/u0-krlmX1_0/hq720.jpg?v=65f4c4d6",
"publish_date": "2022-02-28 00:00:00",     "length": 952,
"author": "Zeeshan Us...
__________________________________________________

In [57]:
response = query_engine.query("tell me about the people of pakistan?")
pprint_response(response,show_source=True)
print(response)

Final Response: According to the given information, Pakistan has a
population of 219 million people. Out of these, 10 million people are
disabled, either handicapped or having some other kind of disability.
There are 40 million people who are below 14 years of age, and 10
million people who are above 65 years old. Additionally, there are
86000 Pakistanis who are imprisoned, 11000 Pakistanis who are
imprisoned in other countries, 160,000 Pakistanis who are suffering
from AIDS, and 17 million people who have Hepatitis B or C.
Furthermore, 32 million people in Pakistan have heart diseases, and
37.5 million people are diabetic.
______________________________________________________________________
Source Node 1/2
Node ID: 907f620b-c0e9-4f43-bb98-5c6915855c48
Similarity: 0.8677311262422863
Text: Pakistan Air Force, Pakistan Navy is a part of Pakistan Army,
which protects all these 219 million population, absolutely inside
Pakistan. According to a rough estimate, there are about 10 million
s